# Image-Bot

## 0. Setup

Setup the relevant variables for your training data generation pipeline. The following variables can be set:

- project_name: The name of the project for which you will generate the training data set. One project contains multiple objects (see object_name), for which training data is generated. For each project a project folder is genearted under "output" to store the object's data.
- object_name: The name of the object you are currently capturing with this jupyter notebook. Each object's data will be placed in a separate folder within your project's folder. If you set the project name and the object name to names which you used previously, you can extend an already existing dataset or process it further.
- object_position: The center of the object, relative to the robot arm in mm.
- arduino_serial_port: The name of the Serial Port to which the Braccio Robot Arm's Arduino is connected. Normally, this is something like 'Com1' (Windows) or '/dev/ttyUSB0' (Ubuntu). If you don't know how to find the correct Serial Port, have a look in the [User Manual](doc/UserManual.md).
- camera: The index of the camera to use. If your computer doesn't have a built-in camera, you might need to put 0 here. If your computer has a built-in camera, the index to use might be most probably 1. Otherwise, just check the values in between 0 and 5 to figure out, which camera is the right one to choose.
- different_backgrounds (optional): Parameter to set with how many different backgrounds each foreground image should be blended. Increase this value if you need more training data and decrease it to have less training data. Must be greater or equal to 1.
- prevent_blurred_objects (optional): Some objects with a very uniform texture might be blurred when blending with the background image. This variable can help to prevent it.

Always run this part before running any other section of this notebook!

To run each part in the following, please click in the code block to run and then on the play icon in the main menu bar on top.

In [ ]:
# Please insert your project and object name here
project_name = "TestPaper"
object_name = "Tissue"
object_position = [600, 0, 50]
arduino_serial_port = "Com1"
camera = 1

# Optional parameters
different_backgrounds = 2
prevent_blurred_objects = True


# ======================================= # Do not alter anything below this line! =======================================
import os
project_folder = os.path.join("./output", project_name)
object_folder = os.path.join(project_folder, object_name)
raw_folder = os.path.join(object_folder, "0_raw")
masked_folder = os.path.join(object_folder, "1_masked")
blended_folder = os.path.join(object_folder, "2_blended")
yolo_folder = os.path.join(object_folder, "3_yolo")

from ImageBot.Config import *
SERIAL_PORT = arduino_serial_port
CAMERA_ID = camera
MODEL_MULTIPLY_MESSAGE_BACKGROUND_ASSIGNMENT = different_backgrounds
PREVENT_BLURRED_OBJECTS = prevent_blurred_objects

print("Finished successfull! Please proceed with the following steps!")

## 0. Adjust Camera Settings

Only call this part if you want to adjust your camera settings. It only works for **Microsoft Windows**. Otherwise, you can skip this section.

In [ ]:
import cv2
import numpy as np
import os

if os.name == 'nt':
    vid = cv2.VideoCapture(1, cv2.CAP_DSHOW)
    vid.set(cv2.CAP_PROP_SETTINGS, 3)
    input("Adjust your camera settings and hit enter to continue")

    print("Camera resolution is:", vid.get(cv2.CAP_PROP_FRAME_WIDTH), "x", vid.get(cv2.CAP_PROP_FRAME_HEIGHT))

    while vid.isOpened():
        ret, img = vid.read()
        cv2.imshow("Image", img)
        if cv2.waitKey(33) & 0xFF == ord('q'):
            break

    vid.release()
    cv2.destroyAllWindows()

## 1. Data Aquisition

This section aquires the raw images of the objects in front of the greenscreen. Please place your object properly and make sure the Arduino for the robot is connected, before starting this section.

The script will control the robot to take pictures of your object from different vertical perspectives. After each run a prompt is shown, which asks, whether you want to rotate the object by hand and do another data aquisition sequence from a different horizontal perspective.
**Make sure to capture enough different perspectives of the object to be recognized!**

In [ ]:
from ImageBot.data_aquisition.DataAquisitionSequence import DataAquistionSequence
import cv2
import numpy as np
import os


def image_taken_clbck(image):
    print("Image %s has been taken!")
    cv2.imshow('bounding box', image)
    # Wait until destruction
    cv2.waitKey(0)
    cv2.destroyAllWindows()


seq = DataAquistionSequence(raw_folder)
seq.init(SERIAL_PORT, CAMERA_ID)

run = True
while run:
    run_next = input("Do you want to run the data aquisition (press enter for yes, any letter for quit):")
    if not run_next:
        seq.aquire(object_position, 50, 300, 5, image_taken_clbck)
    else:
        run = False
        
seq.close()

## 2. Greenscreen Removal

All captured images are shown in sequential order. For each image select a green value via left-click with the mouse in the image. After green value selection, the associated mask is shown. It will disappear after 1 sec or whenever you press any key. After that you can:

- Select a better green value via left mouse click again
- Confirm your green value selection by pressing any key (except 'x')
- Skip this image by pressing the 'x' key (Notice: This image will then not be processed any further)

In [ ]:
from ImageBot.image_processing import PostProcessor as post
from pathlib import Path
from ImageBot.infrastructure.ImageMessage import ImageMessage
from ImageBot.infrastructure.filter import show, load_image
import uuid
from tqdm import tqdm

post.init(dest_folder=Path(masked_folder))
post.load_images(Path(raw_folder))

progress = tqdm(total=post.Loader.qsize())
def clbck(message):
    progress.update()

while not post.Loader.empty():
    result : ImageMessage = ImageMessage(uuid.uuid4())
    load_image(result, post.Loader)
    post.PostProcessor.execute(result, clbck=clbck)

post.PostProcessor.join()

## 3. Image Blending

This part will blend the foreground image with different background images. Just start this process - no interaction is necessary.

**!Warning!** This section uses highly parallelized processing and thus will use 100% of your CPU. Do not start, if you want to do other things with your computer in the meantime. ;-)

*If you don't have a physical setup but want to test the Image-Bot with our datasets, copy the objects in the "output" folder, insert the correct names in Step "0. Setup" and then directly proceed with this step.*

In [ ]:
from ImageBot.data_augmentation import AugmentationPipeline as aug

from pathlib import Path
from ImageBot.infrastructure.ImageMessage import ImageMessage
from ImageBot.infrastructure.filter import show, load_image
import uuid

from tqdm import tqdm

aug.load_images(source_folder=Path(masked_folder), bgs_folder=Path('./bgs/'))
aug.init(dest_folder=Path(blended_folder))

progress = tqdm(total=aug.Loader.qsize())

def clbck(message):
    progress.update()

while not aug.Loader.empty():
    result : ImageMessage = ImageMessage(uuid.uuid4())
    load_image(result, aug.Loader, load_mask=True)
    aug.AugmentationPipeline.execute(result, clbck=clbck)

aug.AugmentationPipeline.join()

## 4. Convert to YOLO dataset

This part converts your generated and labled images to a format which can directly be used to train YOLO image detection algorithms.

In [ ]:
from ImageBot.to_yolo.YoloConverter import to_yolo_dataset

to_yolo_dataset(blended_folder, yolo_folder, test_training_split=0.3)
print("Finished successfully!")

## 5. Train you model

Take the yolo folder for each of your objects and use it in your machine training environment to train your algorithm!
Have fun! :-)